### Import

In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib
import seaborn as sns

# 각 라이브러리 버전 출력
print("Numpy version:", np.__version__)
print("Scikit-learn version:", sklearn.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Numpy version: 1.26.4
Scikit-learn version: 1.5.1
Pandas version: 1.5.3
Matplotlib version: 3.9.2
Seaborn version: 0.13.2


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier

### Data Load

In [3]:
train = pd.read_csv('../../train.csv').drop(columns=['ID'])
test = pd.read_csv('../../test.csv').drop(columns=['ID'])

In [4]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [5]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [6]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)


In [7]:
# 결측값을 채울 칼럼 목록
columns_to_fill = [
    '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
    '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수',
    '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수',
    '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수',
    '대리모 여부'
]

# 시술 유형이 'DI'인 경우에만 결측값을 0으로 채우기
X.loc[X['시술 유형'] == 'DI', columns_to_fill] = X.loc[X['시술 유형'] == 'DI', columns_to_fill].fillna(0)

# 1. '난자 출처'의 결측값을 '본인 제공'으로 채우기
X['난자 출처'].replace('알 수 없음','본인 제공', inplace=True)

# 2. '난자 기증자 나이' 결측값을 시술 당시 나이로 채우기
X.loc[X['난자 기증자 나이'] == '알 수 없음', '난자 기증자 나이'] = X['시술 당시 나이']

# 3. '파트너 정자와 혼합된 난자 수'의 결측값을 0으로 채우기
X['파트너 정자와 혼합된 난자 수'].fillna(0, inplace=True)

# 4. '기증자 정자와 혼합된 난자 수'의 결측값을 1로 채우기
X['기증자 정자와 혼합된 난자 수'].fillna(1, inplace=True)

# 5. '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부'의 결측값을 0으로 채우기
for column in ['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']:
    X[column].fillna(0, inplace=True)

    # 변경 내용 확인
print(X[['난자 출처', '난자 기증자 나이', '파트너 정자와 혼합된 난자 수', 
            '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']].head())


   난자 출처 난자 기증자 나이  파트너 정자와 혼합된 난자 수  기증자 정자와 혼합된 난자 수 동결 배아 사용 여부  \
0  본인 제공   만18-34세               5.0               0.0         0.0   
1  본인 제공   만45-50세               1.0               0.0         0.0   
2  본인 제공   만18-34세               7.0               0.0         0.0   
3  본인 제공   만35-37세               4.0               0.0         0.0   
4  본인 제공   만18-34세               6.0               0.0         0.0   

  신선 배아 사용 여부 기증 배아 사용 여부  
0         1.0         0.0  
1         1.0         0.0  
2         1.0         0.0  
3         1.0         0.0  
4         1.0         0.0  


In [8]:
# 시술 유형이 'DI'인 경우에만 결측값을 0으로 채우기
test.loc[test['시술 유형'] == 'DI', columns_to_fill] = test.loc[test['시술 유형'] == 'DI', columns_to_fill].fillna(0)

# 1. '난자 출처'의 결측값을 '본인 제공'으로 채우기
test['난자 출처'].replace('알 수 없음','본인 제공', inplace=True)

# 2. '난자 기증자 나이' 결측값을 시술 당시 나이로 채우기
test.loc[test['난자 기증자 나이'] == '알 수 없음', '난자 기증자 나이'] = test['시술 당시 나이']

# 3. '파트너 정자와 혼합된 난자 수'의 결측값을 0으로 채우기
test['파트너 정자와 혼합된 난자 수'].fillna(0, inplace=True)

# 4. '기증자 정자와 혼합된 난자 수'의 결측값을 1로 채우기
test['기증자 정자와 혼합된 난자 수'].fillna(1, inplace=True)

# 5. '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부'의 결측값을 0으로 채우기
for column in ['동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부']:
    test[column].fillna(0, inplace=True)

In [9]:
# 각 열의 결측값 개수 확인
missing_values_count = X.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)

결측값이 있는 열과 개수:
임신 시도 또는 마지막 임신 경과 연수    246981
난자 채취 경과일                 57488
난자 해동 경과일                254915
난자 혼합 경과일                 53735
배아 이식 경과일                 43566
배아 해동 경과일                215982
dtype: int64


In [10]:

X['난자 채취 경과일'].fillna(1, inplace=True)
X['난자 해동 경과일'].fillna(0, inplace=True)
X['난자 혼합 경과일'].fillna(0, inplace=True)
X['배아 이식 경과일'].fillna(0, inplace=True)
X['배아 해동 경과일'].fillna(0, inplace=True)

test['난자 채취 경과일'].fillna(1, inplace=True)
test['난자 해동 경과일'].fillna(0, inplace=True)
test['난자 혼합 경과일'].fillna(0, inplace=True)
test['배아 이식 경과일'].fillna(0, inplace=True)
test['배아 해동 경과일'].fillna(0, inplace=True)

In [11]:
# '알 수 없음'이 포함된 열 찾기
unknown_columns = []

for col in X.columns:
    if X[col].astype(str).str.contains('알 수 없음').any():
        unknown_columns.append(col)

# 결과 출력
print("⚠️ '알 수 없음' 값을 포함하는 컬럼들:")
print(unknown_columns)


⚠️ '알 수 없음' 값을 포함하는 컬럼들:
['시술 당시 나이', '배란 유도 유형', '난자 기증자 나이', '정자 기증자 나이']


In [12]:
# 변환 함수 정의 (isinstance 없이 단순화)
def categorize_reason(value):
    if value == "배아 저장용":  # "배아 저장용" 단독일 경우 1
        return 1
    elif "현재 시술용" in value:  # "현재 시술용"이 포함된 경우 1
        return 1
    else:  # 그 외 모든 값은 0
        return 0

X["배아 생성 주요 이유"] = X["배아 생성 주요 이유"].replace("nan", "현재 시술용")
test["배아 생성 주요 이유"] = test["배아 생성 주요 이유"].replace("nan", "현재 시술용")

# "배아 생성 주요 이유" 칼럼 변환 적용
X["배아 생성 주요 이유"] = X["배아 생성 주요 이유"].apply(categorize_reason)
test["배아 생성 주요 이유"] = test["배아 생성 주요 이유"].apply(categorize_reason)



In [13]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [14]:
columns_to_drop = [
        "임신 시도 또는 마지막 임신 경과 연수",
        "PGD 시술 여부",
        "PGS 시술 여부",
        "남성 주 불임 원인",
        "남성 부 불임 원인",
        "불임 원인 - 정자 농도",
        "불임 원인 - 정자 면역학적 요인",
        "불임 원인 - 정자 운동성",
        "불임 원인 - 정자 형태",
        '정자 기증자 나이',
        '배란 유도 유형'
]
X_train_encoded = X_train_encoded.drop(columns = columns_to_drop)    
X_test_encoded = X_test_encoded.drop(columns = columns_to_drop)  

In [15]:
# 병합할 칼럼들
columns_to_merge = [
    '난자 채취 경과일', '난자 해동 경과일', '난자 혼합 경과일',
    '배아 이식 경과일', '배아 해동 경과일'
]

# 새로운 칼럼 생성: 각 경과일의 합
X_train_encoded['총 경과일'] = X_train_encoded[columns_to_merge].sum(axis=1)

# 기존 칼럼 제거
X_train_encoded = X_train_encoded.drop(columns=columns_to_merge)

# 새로운 칼럼 생성: 각 경과일의 합
X_test_encoded['총 경과일'] = X_test_encoded[columns_to_merge].sum(axis=1)

# 기존 칼럼 제거
X_test_encoded = X_test_encoded.drop(columns=columns_to_merge)

In [16]:
# 각 열의 결측값 개수 확인
missing_values_count = X_train_encoded.isnull().sum()

# 결측값이 있는 열만 필터링
missing_columns = missing_values_count[missing_values_count > 0]

print("결측값이 있는 열과 개수:")
print(missing_columns)


결측값이 있는 열과 개수:
Series([], dtype: int64)


In [17]:
X['배아 생성 주요 이유'].value_counts()

1    253108
0      3243
Name: 배아 생성 주요 이유, dtype: int64

In [18]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 52 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   시술 시기 코드          256351 non-null  float64
 1   시술 당시 나이          256351 non-null  float64
 2   시술 유형             256351 non-null  float64
 3   특정 시술 유형          256351 non-null  float64
 4   배란 자극 여부          256351 non-null  float64
 5   단일 배아 이식 여부       256351 non-null  float64
 6   착상 전 유전 검사 사용 여부  256351 non-null  float64
 7   착상 전 유전 진단 사용 여부  256351 non-null  float64
 8   여성 주 불임 원인        256351 non-null  float64
 9   여성 부 불임 원인        256351 non-null  float64
 10  부부 주 불임 원인        256351 non-null  float64
 11  부부 부 불임 원인        256351 non-null  float64
 12  불명확 불임 원인         256351 non-null  float64
 13  불임 원인 - 난관 질환     256351 non-null  float64
 14  불임 원인 - 남성 요인     256351 non-null  float64
 15  불임 원인 - 배란 장애     256351 non-null  float64
 16  불임 원인 - 여성 요인     25

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA



# 데이터 정규화
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X_train_encoded)
scaled_test_x = scaler.transform(X_test_encoded)

# PCA 적용
pca = PCA(n_components=0.95)  # 설명 분산의 93%를 유지하도록 설정
X_train_encoded = pca.fit_transform(scaled_features)
X_test_encoded = pca.transform(scaled_test_x)

# PCA 적용 후 데이터셋의 형태 확인
pca_features_shape = X_train_encoded.shape

# 설명된 분산 비율
explained_variance = pca.explained_variance_ratio_

pca_features_shape, explained_variance

((256351, 29),
 array([0.16753015, 0.11126319, 0.07962371, 0.05178603, 0.04250971,
        0.03946191, 0.03723161, 0.03422201, 0.03335946, 0.03292284,
        0.02821389, 0.02750599, 0.02500275, 0.02422428, 0.0222452 ,
        0.0206592 , 0.01960565, 0.01914876, 0.01862165, 0.01843826,
        0.01761127, 0.01645639, 0.01397385, 0.01081516, 0.01027633,
        0.00865718, 0.00772819, 0.0070885 , 0.00667548]))

### Train

In [20]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

**앙상블 및 Optuna**

In [21]:
import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
import numpy as np

In [22]:
def optimize_xgboost(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5)
    }
    model = XGBClassifier(**params, random_state=42, use_label_encoder=False, eval_metric="logloss")
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")
    return np.mean(scores)

In [23]:
def optimize_lightgbm(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0)
    }
    model = LGBMClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)

In [24]:
def optimize_random_forest(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_samples_split": trial.suggest_int("min_samples_split", 3, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4)
    }
    model = RandomForestClassifier(**params, random_state=42)
    scores = cross_val_score(model, X_train_encoded, y, cv=5, scoring="roc_auc")  
    return np.mean(scores)


In [25]:
# Optuna 스터디 실행
print("Optimizing XGBoost...")
xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(optimize_xgboost, n_trials=30)
best_xgb_params = xgb_study.best_params

print("Optimizing LightGBM...")
lgbm_study = optuna.create_study(direction="maximize")
lgbm_study.optimize(optimize_lightgbm, n_trials=30)
best_lgbm_params = lgbm_study.best_params

print("Optimizing RandomForest...")
rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(optimize_random_forest, n_trials=30)
best_rf_params = rf_study.best_params

[I 2025-02-12 22:41:06,437] A new study created in memory with name: no-name-2b906558-605c-4e68-81ca-c07a6c554d63


Optimizing XGBoost...


c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [22:41:06] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [22:41:09] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [22:41:11] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [22:41:14] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warni

Optimizing LightGBM...
[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009796 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009454 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054554
[LightGBM] [Info] Start training from s

[I 2025-02-12 22:49:49,994] Trial 0 finished with value: 0.7217835367899899 and parameters: {'n_estimators': 425, 'max_depth': 12, 'learning_rate': 0.140427165399226, 'num_leaves': 49, 'subsample': 0.601190463068341}. Best is trial 0 with value: 0.7217835367899899.


...

[I 2025-02-12 22:52:47,548] Trial 28 finished with value: 0.7241128084171654 and parameters: {'n_estimators': 326, 'max_depth': 3, 'learning_rate': 0.029020367735533015, 'num_leaves': 27, 'subsample': 0.8216238406274845}. Best is trial 5 with value: 0.7307207935471567.


[LightGBM] [Info] Number of positive: 52982, number of negative: 152098
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007676 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 205080, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258348 -> initscore=-1.054573
[LightGBM] [Info] Start training from score -1.054573
[LightGBM] [Info] Number of positive: 52983, number of negative: 152098
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009586 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 205081, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258352 -> initscore=-1.054554
[LightGBM] [Info] Start training from score -1.054554
[LightGB

[I 2025-02-12 22:52:52,055] Trial 29 finished with value: 0.7291693184402218 and parameters: {'n_estimators': 267, 'max_depth': 6, 'learning_rate': 0.16482184040983644, 'num_leaves': 11, 'subsample': 0.6315908948175457}. Best is trial 5 with value: 0.7307207935471567.
[I 2025-02-12 22:52:52,056] A new study created in memory with name: no-name-c7c7144f-9342-4ce6-9ea9-56fa05dfccd5


Optimizing RandomForest...


[I 2025-02-12 22:58:20,157] Trial 0 finished with value: 0.6938538892004525 and parameters: {'n_estimators': 253, 'max_depth': 3, 'min_samples_split': 8, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.6938538892004525.
[I 2025-02-12 23:06:28,046] Trial 1 finished with value: 0.719009416789052 and parameters: {'n_estimators': 178, 'max_depth': 7, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.719009416789052.
[I 2025-02-12 23:17:35,828] Trial 2 finished with value: 0.7223265769995271 and parameters: {'n_estimators': 217, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.7223265769995271.
[I 2025-02-12 23:31:54,187] Trial 3 finished with value: 0.7247606539678765 and parameters: {'n_estimators': 254, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}. Best is trial 3 with value: 0.7247606539678765.
[I 2025-02-12 23:45:03,083] Trial 4 finished with value: 0.7191344332466432 and parameters: {'n_es

In [26]:
# 최적화된 모델 생성
xgb_model = XGBClassifier(**best_xgb_params, random_state=42, use_label_encoder=False, eval_metric="logloss")
lgbm_model = LGBMClassifier(**best_lgbm_params, random_state=42)
rf_model = RandomForestClassifier(**best_rf_params, random_state=42)

In [27]:
# XGBoost 모델의 파라미터 확인
print("XGBoost Best Parameters:")
print(xgb_model.get_params())

# LightGBM 모델의 파라미터 확인
print("\nLightGBM Best Parameters:")
print(lgbm_model.get_params())

# RandomForest 모델의 파라미터 확인
print("\nRandomForest Best Parameters:")
print(rf_model.get_params())

XGBoost Best Parameters:
{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.803187142177975, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': 'logloss', 'feature_types': None, 'gamma': 1.747988745807904, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.02995526858218256, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 7, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': 417, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.703232557283515, 'tree_method': None, 'validate_parameters': None, 'verbosity': None, 'use_label_encoder': F

In [28]:
# Soft Voting 앙상블
ensemble_model = VotingClassifier(
    estimators=[
        ("xgb", xgb_model),
        ("lgbm", lgbm_model),
        ("rf", rf_model)
    ],
    voting="soft"
)

In [29]:
# 전체 데이터로 학습
ensemble_model.fit(X_train_encoded, y)

c:\Users\hoyeong\anaconda3\envs\lg_ai\lib\site-packages\xgboost\core.py:158: UserWarning: [03:17:47] WARNING: D:\bld\xgboost-split_1722454234704\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 66228, number of negative: 190123
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007583 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 256351, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258349 -> initscore=-1.054568
[LightGBM] [Info] Start training from score -1.054568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.803187142177975,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            gamma=1.747988745807904,
                                            grow_policy=None,
                                            importance_type=None,
                                            interacti...
                                            multi_strategy=None,
                                            n_estimators=417, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('lgbm',
                              LGBMClassifier(learning_rate=0.05282341078677656,
                                             max_depth=5, n_estimators=291,
                                             num_leaves=45, random_state=42,
                                             subsample=0.8019509732556491)),
                             ('rf',
                              RandomForestClassifier(max_depth=12,
                                                     min_samples_split=4,
                                                     n_estimators=155,
                                                     random_state=42))],
                 voting='soft')

### Predict

In [30]:
pred_proba = ensemble_model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [31]:
sample_submission = pd.read_csv('../../sample_submission.csv')
sample_submission['probability'] = pred_proba

In [32]:
sample_submission.to_csv('./Result/baseline_submit.csv', index=False)

In [33]:
pred_proba

array([0.00678237, 0.01148497, 0.16466486, ..., 0.43074943, 0.31741161,
       0.02870452])

In [34]:
unique, counts = np.unique(sample_submission['probability'], return_counts=True)
print(unique, counts)

[3.01266463e-04 3.12348210e-04 3.18056581e-04 ... 7.16311330e-01
 7.23030638e-01 7.38325415e-01] [1 1 1 ... 1 1 1]
